In [1]:
#define main function block
from mxnet.gluon import nn

def mlpconv(channels,kernel_size,padding,strides,max_pooling=True):
    out=nn.Sequential()
    out.add(
        nn.Conv2D(channels=channels,kernel_size=kernel_size,strides=strides,padding=padding,activation='relu'),
        nn.Conv2D(channels=1,kernel_size=1,strides=1,padding=0,activation='relu'),
        nn.Conv2D(channels=1,kernel_size=1,strides=1,padding=0,activation='relu'))
    if max_pooling:#为什么会有maxpooling？
        out.add(nn.MaxPool2D(pool_size=3, strides=2))
    return out


#batch x channel x height x width

In [2]:
net=nn.Sequential()
#kernel1: 11×11, channels: 96
#kernel2: 5×5, channels: 256
#kernel3: 3×3, channels: 384

with net.name_scope():
    net.add(
        mlpconv(96,11,0,strides=4),
        mlpconv(256,5,2,strides=1),#padding数怎么确定？
        mlpconv(384,3,1,strides=1),#padding数怎么确定？
        nn.Dropout(0.5),
        #最后输出10类，所以是10个通道
        mlpconv(10, 3, 1,strides=1, max_pooling=False),
        nn.AvgPool2D(pool_size=5),
        nn.Flatten()
    )

In [3]:
#train data
from mxnet import gluon
import sys
sys.path.append('..')
import utils
from mxnet import init

train_data,test_data=utils.load_data_fashion_mnist(
batch_size=64,resize=224)

ctx=utils.try_gpu()
net.initialize(ctx=ctx,init=init.Xavier())#使用Xavier来初始化参数

loss=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})

utils.train(train_data,test_data,net,loss,trainer,ctx,num_epochs=1)


ModuleNotFoundError: No module named 'utils'

ModuleNotFoundError: No module named 'utils'